# 设置参数
配置的参数,需要与采集数据时雷达配置的参数一致

In [1]:
import torch
import numpy as np
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from RealTimeCollector.RealTimeCollector import RealTimeCollector
import os
import shutil
import random

n_RX = 4  # RX天线通道总数
n_samples = 64  # 采样点数
n_chirps = 255  # 每帧脉冲数
N = 64  # 距离向FFT点数
M = 64  # 多普勒向FFT点数
Q = 64  # 角度FFT

c = 3.0e8   # 光速
B = 3999.48e6  # 调频带宽
K = 99.987e12  # 调频斜率
T = B / K  # 调频周期
Tc = 140e-6  # chirp总周期
fs = 2.5e6  # 采样率
f0 = 77e9  # 初始频率
lambda_val = c / f0  # 雷达信号波长
d = lambda_val / 2  # 天线阵列间距

numADCBits = 16 # ADC位数
numLanes = 4    # 通道数
isReal = 0      # 是否为实数

# 是否静态杂波滤除
is_static_clutter_removal = 1

num_classes = 5 # 类别数
num_samples = 1600
n_frames = 30

distance_axis = torch.arange(0, N) * fs * c / (2 * K * N)   # 距离轴
velocity_axis = torch.arange(-M/2, M/2) * lambda_val / Tc / M / 2   # 速度轴

rtc = RealTimeCollector(ip_address='127.0.0.50')

In [9]:
# 从指定文件夹中随机复制指定数量的文件到目标文件夹
import os
import shutil
import random
import glob

def copy_random_files(source_path, target_path, class_list, num_files_per_class,cnt=0):

    # 创建目标类别文件夹
    target_dir = os.path.join(target_path, '9')
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 遍历指定的类别
    for label in class_list:
        print(f'当前类别为：{label}')
        source_dir = os.path.join(source_path, str(label))

        bin_files = sorted(glob.glob(os.path.join(source_dir, '*.bin')))

        # 随机抽取指定数量的文件
        random_files = random.sample(bin_files, num_files_per_class)

        # 复制文件到目标文件夹并重命名
        for i, file in enumerate(random_files):
            cnt += 1
            shutil.copy(file, os.path.join(target_dir, f'{cnt}.bin'))

# 使用示例
source_path = 'K:/dataset/2023_12_7/'
target_path = 'K:/temp/AA/'
class_list = [5, 6, 7, 8]  # 需要提取的类别列表
num_files_per_class = 200  # 每个类别随机复制200个文件
copy_random_files(source_path, target_path, class_list, num_files_per_class, cnt=0)


当前类别为：1
当前类别为：2
当前类别为：3
当前类别为：4


In [2]:
# 将指定文件夹下的所有文件夹中的所有.bin文件重命名为1到n_samples,并删除.txt和.csv文件
import os
import glob

def rename_and_remove_files(file_path, num_classes):
    for clss in range(1, num_classes + 1):
        file_dir = os.path.join(file_path, str(clss))
        print(file_dir)

        # 获取目录下所有文件名
        all_files = sorted(os.listdir(file_dir))

        # 重命名.bin文件
        cnt_bin = 1
        for file_name in all_files:
            file_path_old = os.path.join(file_dir, file_name)
            file_extension = os.path.splitext(file_name)[1]

            if file_extension == '.bin':
                file_name_new = f'{cnt_bin}.bin'
                file_path_new = os.path.join(file_dir, file_name_new)

                # 如果新的文件名已存在，则加上序号后缀
                while os.path.exists(file_path_new):
                    cnt_bin += 1
                    file_name_new = f'{cnt_bin}.bin'
                    file_path_new = os.path.join(file_dir, file_name_new)

                os.rename(file_path_old, file_path_new)
                cnt_bin += 1

        # 删除.txt和.csv文件
        for file_name in all_files:
            file_path = os.path.join(file_dir, file_name)
            file_extension = os.path.splitext(file_name)[1]

            if file_extension in ['.txt', '.csv']:
                os.remove(file_path)

file_path = 'K:/dataset/1000'
num_classes = 1  # 假设有10个类别
rename_and_remove_files(file_path, num_classes)


K:/dataset/1000\1


In [7]:
# 用于合并多个.mat文件
import os
import scipy.io as sio
import numpy as np

def load_mat_files(input_path):
    data = sio.loadmat(input_path)
    range_profile = data['range_profile']
    speed_profile = data['speed_profile']
    angle_profile = data['angle_profile']
    return range_profile, speed_profile, angle_profile

# 定义合并.mat文件的函数
def merge_mat_files(input_path_list, output_path):
    range_profile = []
    speed_profile = []
    angle_profile = []
    
    for input_path in input_path_list:
        range_profile_, speed_profile_, angle_profile_ = load_mat_files(input_path)
        range_profile.append(range_profile_)
        speed_profile.append(speed_profile_)
        angle_profile.append(angle_profile_)
    
    range_profile = np.concatenate(range_profile, axis=0)
    speed_profile = np.concatenate(speed_profile, axis=0)
    angle_profile = np.concatenate(angle_profile, axis=0)
    
    
    sio.savemat(output_path, {'range_profile': range_profile, 'speed_profile': speed_profile, 'angle_profile': angle_profile})

# 定义合并数据集的函数
def merge_datasets(input_path_list, output_path, num_classes):
    for i in range(1, num_classes+1):
        input_path = [os.path.join(path, str(i)+'.mat') for path in input_path_list]
        merge_mat_files(input_path, os.path.join(output_path, f'{i}.mat'))

input_path_list = ['K:/aio_radar/out/1000/avg_remove', 'K:/aio_radar/out/1000/avg']
out_path = 'K:/aio_radar/out/1000/test'
merge_datasets(input_path_list, out_path, 4)